In [1]:
import os
import sys
import subprocess as sp
import json
import random
import time
import hashlib
import base64
from contextlib import closing

import multiprocessing
from multiprocessing import Pool

from concurrent.futures.thread import ThreadPoolExecutor
from concurrent.futures.process import ProcessPoolExecutor

import psycopg2
import psycopg2.extras


### Get a quick pg instance:
---

```bash
# ephemeral pg13
docker run --rm -it --name pg13 --network host -e POSTGRES_PASSWORD=bobz1234 postgres:13 --port=9932


# put data under tmpfs
# PGDATA tells it where to put data, needs to be someplace that can be chowned to postgres user
# by default things should be under /var/lib/postgresql/data
docker run --rm -it --name pg13 --network host -e POSTGRES_PASSWORD=bobz1234 \
-e PGDATA=/var/lib/postgresql/data/tmpfs_vol/pgdata --tmpfs /var/lib/postgresql/data/tmpfs_vol/ postgres:13 --port=9932


# if connecting from remote add
--listen-addresses='*'
```

In [2]:

# dsn = "... dbname=postgres options='-c synchronous_commit=off'"
_PG_DSN_SYNC = {
    # "host": "192.168.24.141",
    # "port": "5432",
    
    "host": "127.0.0.1",
    "port": "9932",
    
    "database": "postgres",
    "user": "postgres",
    "password": 'bobz1234',
    
    # any other options should be passed along to underlying lib, but only those psycopg2 understands
    # and accepts. "nonz": "hello world" wouldnt be accepted or passed along
    "application_name": "pg_bench_ntbk",
}

_PG_DSN_NO_SYNC = _PG_DSN_SYNC.copy() # shallow is enof.
_PG_DSN_NO_SYNC["options"] = "-c synchronous_commit=off"
_PG_DSN_NO_SYNC["application_name"] = "fts_ntbk_unsync"

In [3]:
sync_dbconn = psycopg2.connect(**_PG_DSN_SYNC)
sync_dbconn.autocommit = True

unsync_dbconn = psycopg2.connect(**_PG_DSN_NO_SYNC)
unsync_dbconn.autocommit = True

In [4]:
sync_cur = sync_dbconn.cursor()
unsync_cur = unsync_dbconn.cursor()

In [5]:
# common q exec.
def exec_q_unsync(q, pr_res=True, col_names=True):
    with closing(unsync_dbconn.cursor()) as tmp_cur:
        tmp_cur.execute(q)
        if pr_res:
            if col_names:
                print("|".join([desc[0] for desc in tmp_cur.description]))
            print('-'*40)
            for row in tmp_cur.fetchall():
                print(row)
            # sep
            print("")
# sync
def exec_q_sync(q, pr_res=True, col_names=True):
    with closing(sync_dbconn.cursor()) as tmp_cur:
        tmp_cur.execute(q)
        if pr_res:
            if col_names:
                print("|".join([desc[0] for desc in tmp_cur.description]))
            print('-'*40)
            for row in tmp_cur.fetchall():
                print(row)
            # sep
            print("")

# Run some test queries

In [6]:
exec_q_sync("SHOW synchronous_commit;")
exec_q_unsync("SHOW synchronous_commit;")

synchronous_commit
----------------------------------------
('on',)

synchronous_commit
----------------------------------------
('off',)



# Generate sample 

In [7]:
# mariadb INT type is ranged: -2 147 483 648 to 2 147 483 647
def get_random_int32():
    tmp = 2 * 1000 * 1000 * 1000
    return random.randint(0, tmp)

def get_random_int64():
    tmp = 80 * 1000 * 1000 * 1000 * 1000
    return random.randint(0, tmp)

def get_random_str(str_len=48):
    assert isinstance(str_len, int)
    assert str_len < 8192

    return base64.b64encode(os.urandom(str_len), altchars=b"AZ").decode('ascii')

In [8]:
tmp_table = []
for _ in range(5):
    tmp_table.append((get_random_str(48), get_random_int32()))

for row in tmp_table:
    print(row)

('SHiKQb2PtvEJFmomPtQjXzhbZqrMdTXFwLL5XcwenZzfr4tk86W9vFmIk2pzxsjD', 622804870)
('mKwRXmoddUqnAo1GqS0bwIVZ29xSKcqfNdlNoSd7YaJ7ZTgkWWMfpwyKbuDLpXRN', 1615188628)
('6K4CQ3cAwI44uAXmgV4wDzr2iErcBmCAK7Q4Hq7PEqvvpZsAW4TE5Vb9vExYMfQi', 1442077397)
('o2P5cIkHrxArO1w9W9JqtCthpsi0Al0mCMVWrAZqUQdle7xxwHlUAFgO2l4E2wkQ', 909154400)
('EZZ9ESdzayTlPwy06egJCROxqehi9pipkLj3cFPTeaL0re6cieA5OV6T9d0sUuQC', 1433974176)


In [9]:
len('Q8M5rzhQveJJClZAGhXtNmWCgRm2ETnSeZb2KdXy1vDVlnj3tYOlOZulh2Fc7T1p')

64

# Create BENCHMARK tables

In [10]:
refresh_schema_q = """
DROP TABLE IF EXISTS xb_bench1 CASCADE;
DROP TABLE IF EXISTS xb_bench2 CASCADE;

CREATE TABLE IF NOT EXISTS xb_bench1(
brid SERIAL PRIMARY KEY NOT NULL,
val_1 TEXT,
num_1 BIGINT);


CREATE TABLE IF NOT EXISTS xb_bench2(
brid SERIAL PRIMARY KEY NOT NULL,
val_1 TEXT,
num_1 BIGINT);
"""

In [11]:
exec_q_sync(refresh_schema_q, pr_res=False)

In [12]:
# test insert some records

In [13]:
q = """ INSERT INTO xb_bench1(val_1, num_1) VALUES 
('Q8M5rzhQveJJClZAGhXtNmWCgRm2ETnSeZb2KdXy1vDVlnj3tYOlOZulh2Fc7T1p', '2120724'),
('Q8M5rzhQveJJClZAGhXtNmqCgRm2ETnSeZb2KdXy3vDVlnj3tYOlOZulh2Fv7T1p', '2020921');
"""

sync_cur.execute(q)

In [14]:
exec_q_sync("select * from xb_bench1;")

brid|val_1|num_1
----------------------------------------
(1, 'Q8M5rzhQveJJClZAGhXtNmWCgRm2ETnSeZb2KdXy1vDVlnj3tYOlOZulh2Fc7T1p', 2120724)
(2, 'Q8M5rzhQveJJClZAGhXtNmqCgRm2ETnSeZb2KdXy3vDVlnj3tYOlOZulh2Fv7T1p', 2020921)



In [15]:
q = "INSERT INTO xb_bench1(val_1, num_1) VALUES (%s, %s);"
record = ('onuqwddqobqubffqubf', 13400111)
sync_cur.execute(q, record)

In [16]:
exec_q_sync("select * from xb_bench1;")

brid|val_1|num_1
----------------------------------------
(1, 'Q8M5rzhQveJJClZAGhXtNmWCgRm2ETnSeZb2KdXy1vDVlnj3tYOlOZulh2Fc7T1p', 2120724)
(2, 'Q8M5rzhQveJJClZAGhXtNmqCgRm2ETnSeZb2KdXy3vDVlnj3tYOlOZulh2Fv7T1p', 2020921)
(3, 'onuqwddqobqubffqubf', 13400111)



In [17]:
exec_q_sync(refresh_schema_q, pr_res=False)

In [18]:
exec_q_sync("select * from xb_bench1;")

brid|val_1|num_1
----------------------------------------



# BENCHMARK

In [19]:
# ---------------------------------------- sync execute
num_records = 40 * 1000

# drop tables and start fresh
tmp_cur = sync_dbconn.cursor()
tmp_cur.execute(refresh_schema_q)

# sample data
tmp_table = []
for _ in range(num_records):
    tmp_table.append((get_random_str(48), get_random_int32()))

print("sample data rdy ...")

# query
q = "INSERT INTO xb_bench1(val_1, num_1) VALUES (%s, %s);"

# *** time insertion
start_time = time.perf_counter()
for record in tmp_table:
    tmp_cur.execute(q, record)

# stop time
elapsed_time = time.perf_counter() - start_time

print(f"time: {elapsed_time}")
print(f"ips: {num_records/elapsed_time}")

tmp_cur.execute("select count(*) from xb_bench1;")
print(f"\nActual count(*): {tmp_cur.fetchone()}")

tmp_cur.close()

sample data rdy ...
time: 3.4433179199986625
ips: 11616.702532078576

Actual count(*): (40000,)


In [20]:
# ---------------------------------------- unsync execute
num_records = 40 * 1000

# drop tables and start fresh
tmp_cur = unsync_dbconn.cursor()
tmp_cur.execute(refresh_schema_q)

# sample data
tmp_table = []
for _ in range(num_records):
    tmp_table.append((get_random_str(48), get_random_int32()))

print("sample data rdy ...")

# query
q = "INSERT INTO xb_bench1(val_1, num_1) VALUES (%s, %s);"

# *** time insertion
start_time = time.perf_counter()
for record in tmp_table:
    tmp_cur.execute(q, record)

# stop time
elapsed_time = time.perf_counter() - start_time

print(f"time: {elapsed_time}")
print(f"ips: {num_records/elapsed_time}")

tmp_cur.execute("select count(*) from xb_bench1;")
print(f"\nActual count(*): {tmp_cur.fetchone()}")

tmp_cur.close()

sample data rdy ...
time: 3.4253708310134243
ips: 11677.567765171185

Actual count(*): (40000,)


In [21]:
# ---------------------------------------- sync fast bulk load
num_records = 90 * 1000

# drop tables and start fresh
tmp_cur = sync_dbconn.cursor()
tmp_cur.execute(refresh_schema_q)

# sample data
tmp_table = []
for _ in range(num_records):
    tmp_table.append((get_random_str(48), get_random_int32()))

print("sample data rdy ...")

# query
q = "INSERT INTO xb_bench1(val_1, num_1) VALUES %s;"

# *** time insertion
start_time = time.perf_counter()
psycopg2.extras.execute_values(tmp_cur, q, tmp_table)
elapsed_time = time.perf_counter() - start_time


print(f"time: {elapsed_time}")
print(f"ips: {num_records/elapsed_time}")

tmp_cur.execute("select count(*) from xb_bench1;")
print(f"\nActual count(*): {tmp_cur.fetchone()}")

tmp_cur.close()

sample data rdy ...
time: 0.8536967730033211
ips: 105423.84936442751

Actual count(*): (90000,)


In [22]:
# ---------------------------------------- unsync fast bulk load
num_records = 90 * 1000

# drop tables and start fresh
tmp_cur = unsync_dbconn.cursor()
tmp_cur.execute(refresh_schema_q)

# sample data
tmp_table = []
for _ in range(num_records):
    tmp_table.append((get_random_str(48), get_random_int32()))

print("sample data rdy ...")

# query
q = "INSERT INTO xb_bench1(val_1, num_1) VALUES %s;"

# *** time insertion
start_time = time.perf_counter()
psycopg2.extras.execute_values(tmp_cur, q, tmp_table)
elapsed_time = time.perf_counter() - start_time


print(f"time: {elapsed_time}")
print(f"ips: {num_records/elapsed_time}")

tmp_cur.execute("select count(*) from xb_bench1;")
print(f"\nActual count(*): {tmp_cur.fetchone()}")

tmp_cur.close()

sample data rdy ...
time: 0.8280718580062967
ips: 108686.21983687271

Actual count(*): (90000,)


---
---
---
# pc1 results
* ryzen 2700x
* samsung nvme 970 evo 500GB
* 32 GB DDR4
* Ubuntu 18.04 (bionic)

---
### tmpfs data volume


|                insertion method                  |   ips (dkr/pg13)   |
|:-------------------------------------------------|:------------------:|
| regular execute                                  |        12 K        | 
| regular execute,         no sync                 |        12 K        |
| extras.execute_values                            |       110 K        |
| extras.execute_values,   no sync                 |       110 K        |


---
---
---
# Raspberry pi 3 B
* Quad core A53
* 128GB micro sd,
* kernel 5.4, ubuntu server,
* docker pg13

# pgdata on tmpfs

|                insertion method                  | ips (pi3/dkr/pg13) |
|:-------------------------------------------------|:------------------:|
| regular execute                                  |      ~ 1000        | 
| regular execute,         no sync                 |      ~ 1000        |
| extras.execute_values                            |      ~ 15 K        |
| extras.execute_values,   no sync                 |      ~ 15 K        |


# 128GB sandisk sd card, raspberry pi 3 B 


|                insertion method                  | ips (pi3/dkr/pg13) | ips (pi3/pg11) |
|:-------------------------------------------------|:------------------:|:--------------:|
| regular execute                                  |     ~ 380          |   ~ same       |
| regular execute,         no sync                 |      1200          |    950         |
| extras.execute_values                            |         7 K        |   ~ same       |
| extras.execute_values,   no sync                 |        16 K        |      14 K      |


# crap 8 gb kingston sd card

```text

|                insertion method                  | ips (pi3/dkr/pg13) |
|:-------------------------------------------------|:------------------:|
| regular execute                                  |       100          |
| regular execute,         no sync                 |       800          |
| extras.execute_values                            |       600          |
| extras.execute_values,   no sync                 |      8 K           |

Note1: w/ crap kingston sd "exec no sync" does better than "sync bulk insert" (800 vs 600)
"no sync bulk insert" does 8k of course. showing how clearly kingston is the bottleneck.

we are testomg raw ips, this was always testing disk in sync mode.
its not sysbench OLTP, no lock contention or anything like that.

```

---
---
---
# orange pi one:
* allwinner H3 quad core (3 enabled) armv7lhf,
* 512mbddr3,
* 16gb microsd lexxar,
* kernel 3.4, pg9.3
* ubuntu 14 (trusty) provided by orange pi w/ custom kernel. not upstream.

```text

|                insertion method                  | ips (orange pi one/pg9.3) |
|:-------------------------------------------------|:-------------------------:|
| regular execute                                  |       ~ 200               |
| regular execute,         no sync                 |        1500               |
| extras.execute_values                            |         6 K               |
| extras.execute_values,   no sync                 |        11 K               |

```